In [ ]:
from fh2 import FH2 
import fh2
import copy
import pickle
from LFOFM import *
import PydalChanel as pydalmodule
from OSC import *
import copy

wp = WavePlayer()
lp = FH2()
    

def sendMsgSC(addr, *args):
    msg = OSCMessage()
    msg.setAddress(addr)
    msg.append(args)
    wp.superColliderClient.send(msg)

flatten = lambda l: [item for sublist in l for item in sublist]
buttonToLoopInd = flatten([[(j+1)*10+(i+1) for i in range(8)] for j in range(2, 8)])

plotWave = wp.plotWave
read = pydalmodule.read
callbacks = lp.buttonForwardingHandlers

pydal = pydalmodule.getPydalInstance()

end = pydal.end
tempo = pydal.setTempo
stripSymbols = lambda beats : map(lambda beat : list(beat[1])[0], beats)

ch1 = pydal.newChannel(1)
ch2 = pydal.newChannel(2)
ch3 = pydal.newChannel(3)
ch4 = pydal.newChannel(4)
ch5 = pydal.newChannel(5)
ch5 = pydal.newChannel(5)

def getLoop(button):
    ind = buttonToLoopInd.index(button) + 1
    return copy.deepcopy(lp.loops[ind])

def setLoop(loop, button):
    ind = buttonToLoopInd.index(button) + 1
    lp.loops[ind] = loop
    sendMsgSC("/setBankMelody", lp.hitListToString(loop, button, 0))

def startWave(wave, chan):
    wp.startWaveOSC("chan"+str(chan), "/changeChannelTempo", wave, "sc", chan)
def stopWave(chan):
    wp.stopWave(chan)
    
def speedScale(loop, scaleVal):
    return [[hit[0]*(1./scaleVal)] + hit[1:] for hit in loop]

def setVal(loop, ccVal):
    loop2 = copy.deepcopy(loop)
    for i in range(len(loop)):
        loop2[i][1] = ccVal
    return loop2

def reverse(loop):
    loop2 = copy.deepcopy(loop)
    loop2.reverse()
    loop2[-1][4] = 'timeAfterLastHit'
    loop2[0][4] = 'cc'
    return loop2

def flip(loop):
    loop2 = copy.deepcopy(loop)
    loopVals = [hit[2] for hit in loop2]
    for i in range(len(loop)):
        loop2[i][2] = 127 - loopVals[i]
    return loop2

def getCC(button):
    ind = buttonToLoopInd.index(button) + 1
    return CCLoop(copy.deepcopy(lp.loops[ind]))

class CCLoop:
    
    def __init__(self, loop):
        self.loop = loop
    
    def speedScale(self, scaleVal):
        loop = self.loop
        return CCLoop([[hit[0]*(1./scaleVal)] + hit[1:] for hit in loop])

    def setVal(self, ccVal):
        loop = self.loop
        loop2 = copy.deepcopy(loop)
        for i in range(len(loop)):
            loop2[i][1] = ccVal
        return CCLoop(loop2)

    def reverse(self):
        loop = self.loop
        loop2 = copy.deepcopy(loop)
        loop2.reverse()
        loop2[-1][4] = 'timeAfterLastHit'
        loop2[0][4] = 'cc'
        return CCLoop(loop2)

    def flip(self):
        loop = self.loop
        loop2 = copy.deepcopy(loop)
        loopVals = [hit[2] for hit in loop2]
        for i in range(len(loop)):
            loop2[i][2] = 127 - loopVals[i]
        return CCLoop(loop2)
       
    def setLoop(self, button):
        loop = self.loop
        ind = buttonToLoopInd.index(button) + 1
        lp.loops[ind] = loop
        sendMsgSC("/setBankMelody", lp.hitListToString(loop, button, 0))
    
    def setDrum(self, drumInd):
        loop = self.loop
        loop2 = copy.deepcopy(loop)
        for i in range(len(loop)):
            loop2[i][1] = drumInd*10 + loop2[i][1]%10
        return CCLoop(loop2)
    
pi=3.14159    
# plotWave(w2, 0, 100)

#need to start SC files masterTempoClock, PydalSC, and LFOFM, sceneBuilderMini

## RHYTHM WARP SETUP

In [ ]:
tempo(100)

In [ ]:
ch1.play(read("bd [bd bd]"))

In [ ]:
ch2.play(read("hc [hc hc]"))

In [ ]:
ch3.play(read("[lt lt] lt"))

In [ ]:
sendMsgSC("/syncClocks", "both", 'master', 1, 2, 3)

In [ ]:
ch1.stop()
ch2.stop()
ch3.stop()

In [ ]:
stopWave("chan1")
stopWave("chan2")
stopWave("chan3")

## Talk Cells

In [ ]:
ch1.play(read("bd"))
# ch1.stop()

In [ ]:
startWave(120 + Sin(freq=0.2)*40, 1)
# stopWave(1)
# def startWave(wave, chan):
#     #waveKey, destAddress, wave, OSCPort, meta-info
#     wp.startWaveOSC("chan"+str(chan), "/changeChannelTempo", wave, "sc", chan)
# sendMsgSC("/syncClocks", "both", 'master', 1, 2, 3)

In [ ]:
cc81 = getCC(81)
cc81.reverse().speedScale(2).setVal(1).setLoop(82)